In [ ]:
#pip install bertopic
#pip install --upgrade jupyter ipywidgets
#pip install PyMuPDF bertopic sklearn

In [1]:
import fitz  # PyMuPDF
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from langchain.text_splitter import TokenTextSplitter
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [6]:
# Function to enrich metadata and log history
def enrich_metadata(pdf_path):
    # Extract text from the PDF
    pdf_text = extract_text_from_pdf(pdf_path)
    
    # Initialize the TokenTextSplitter with desired chunk size and overlap
    text_splitter = TokenTextSplitter(chunk_size=50, chunk_overlap=0)
    chunks = text_splitter.split_text(pdf_text)
    
    # Initialize the CountVectorizer with stop words removal
    vectorizer_model = CountVectorizer(stop_words="english")
    
    # Fit the vectorizer on the text to learn the vocabulary
    vectorizer_model.fit([pdf_text])
    
    # Initialize the TokenTextSplitter with a smaller chunk size and overlap
    text_splitter = TokenTextSplitter(chunk_size=50, chunk_overlap=0)
    chunks = text_splitter.split_text(pdf_text)
    
    # Create and fit the BERTopic model using the CountVectorizer
    topic_model = BERTopic(vectorizer_model=vectorizer_model)
    topics, probs = topic_model.fit_transform(chunks)
    
    # Get information about the identified topics
    topic_info = topic_model.get_topic_info()
    
    # Metadata for the PDF document
    title = 'The Solar System'
    author = 'xyz'
    url = 'http://localhost:8888/edit/Documents/Python/solarsystem.pdf'
    
    # Log metadata and topics into a list
    keys = []
    for i in range(len(chunks)):
        text = chunks[i]
        
        keys.append({
            'document': pdf_path,
            'title': title,
            'author': author,
            'url': url,
            'doc': text,
            'topics': topic_info
        })
        
    return keys

In [7]:
# Define the writehistory function 
def writehistory(logging_text):
    with open("history.log", "a", encoding="utf-8") as log_file:
        log_file.write(logging_text + "\n")

In [8]:
# Example usage
pdf_path = '/Users/pradhikshasuresh/Documents/Python/The Solar System.pdf'
enriched_metadata = enrich_metadata(pdf_path)
print(enriched_metadata[2])  # Print metadata and keywords for the third chunk

{'document': '/Users/pradhikshasuresh/Documents/Python/The Solar System.pdf', 'title': 'The Solar System', 'author': 'xyz', 'url': 'http://localhost:8888/edit/Documents/Python/solarsystem.pdf', 'doc': " and light to support life on Earth. It is by far the largest object in the solar system, containing more \nthan 99.8% of the total mass. The Sun's immense gravity keeps the planets and other objects in orbit around \n", 'topics':    Topic  Count                         Name  \
0     -1     16  -1_planet_planets_solar_sun   

                                      Representation  \
0  [planet, planets, solar, sun, jupiter, dwarf, ...   

                                 Representative_Docs  
0  [planets, their moons, dwarf planets, and coun...  }
